In [1]:
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory
import pandas as pd

In [2]:
# inputs
dataGen = pd.read_excel('10_Pyomo_Summations_Data.xlsx', sheet_name='gen')
dataLoad = pd.read_excel('10_Pyomo_Summations_Data.xlsx', sheet_name='load')

In [3]:
dataGen

,id,limit,cost
0,0,20,0.10
1,1,10,0.05
2,2,40,0.30
3,3,50,0.40
4,4,5,0.01


In [4]:
dataLoad

,id,value
0,0,50
1,1,20
2,2,30


In [5]:
Ng = len(dataGen)

In [6]:
model = pyo.ConcreteModel()

model.Pg = pyo.Var(range(Ng), bounds=(0,None)) # 5개 Var 생성(Ng=5)
#참고. 아래 Command는 5x3 Dimension의 Var 생성 가능하다.
#model.Pg = pyo.Var(range(Ng), range(3), bounds=(0,None))

In [7]:
model.Pg.pprint()

Pg : Size=5, Index=Pg_index
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      0 :     0 :  None :  None : False :  True :  Reals
      1 :     0 :  None :  None : False :  True :  Reals
      2 :     0 :  None :  None : False :  True :  Reals
      3 :     0 :  None :  None : False :  True :  Reals
      4 :     0 :  None :  None : False :  True :  Reals


In [8]:
Pg = model.Pg

In [9]:
[Pg[g] for g in dataGen['id']]

In [10]:
print(sum([Pg[g] for g in dataGen['id']]))

Pg[0] + Pg[1] + Pg[2] + Pg[3] + Pg[4]


In [11]:
pg_sum = sum([Pg[g] for g in dataGen['id']])

In [12]:
# dataLoad['value']는 최적화 Variable이 아니므로 그냥 쓸 수 있음
model.balance = pyo.Constraint(expr = pg_sum == sum(dataLoad['value']))

In [13]:
model.cond = pyo.Constraint(expr = Pg[0] + Pg[3] >= dataLoad['value'][0])

In [15]:
model.limits = pyo.ConstraintList()
for g in dataGen['id']:
    model.limits.add(expr = Pg[g] <= dataGen.limit[g])

In [19]:
# objFcn
model.obj = pyo.Objective(expr = sum([Pg[g]*dataGen['cost'][g] for g in dataGen['id']]))

In [20]:
model.pprint()

2 Set Declarations
    Pg_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}
    limits_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}

1 Var Declarations
    Pg : Size=5, Index=Pg_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True :  Reals
          1 :     0 :  None :  None : False :  True :  Reals
          2 :     0 :  None :  None : False :  True :  Reals
          3 :     0 :  None :  None : False :  True :  Reals
          4 :     0 :  None :  None : False :  True :  Reals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : 0.1*Pg[0] + 0.05*Pg[1] + 0.3*Pg[2] + 0.4*Pg[3] + 0.01*Pg[4]

3 Constraint Declarations
    balance :

In [25]:
opt = SolverFactory('cbc', executable='D:\\Solver-CBC\\win32-msvc9\\bin\\cbc.exe')
results = opt.solve(model)

In [27]:
dataGen['Pg'] = [pyo.value(Pg[g]) for g in dataGen['id']]

In [28]:
dataGen

,id,limit,cost,Pg
0,0,20,0.10,20.0
1,1,10,0.05,10.0
2,2,40,0.30,35.0
3,3,50,0.40,30.0
4,4,5,0.01,5.0
